T.U.I.A 2024 - Aprendizaje Automático II 

TP 1 - Redes Densas y Convolucionales

Ponce, Daniel - Yañez, Mirian

Problema 2

Instalamos las dependencias

In [1]:
!pip install mediapipe

  Obtaining dependency information for mediapipe from https://files.pythonhosted.org/packages/f2/99/688a43ae7023c068b18d4bfde91a5df36fb3ae312e2516ca2ec1e8a0b3af/mediapipe-0.10.15-cp310-cp310-macosx_11_0_x86_64.whl.metadata
  Obtaining dependency information for jax from https://files.pythonhosted.org/packages/06/f3/c499d358dd7f267a63d7d38ef54aadad82e28d2c28bafff15360c3091946/jax-0.4.34-py3-none-any.whl.metadata
  Obtaining dependency information for jaxlib from https://files.pythonhosted.org/packages/24/31/2e254fe2fc23201775a7d0ccd1bcde892cfa349eb805744b81b15e0dcf74/jaxlib-0.4.34-cp310-cp310-macosx_10_14_x86_64.whl.metadata
  Obtaining dependency information for opencv-contrib-python from https://files.pythonhosted.org/packages/09/94/d077c4c976c2d7a88812fd55396e92edae0e0c708689dbd8c8f508920e47/opencv_contrib_python-4.10.0.84-cp37-abi3-macosx_12_0_x86_64.whl.metadata
  Obtaining dependency information for sounddevice>=0.4.4 from https://files.pythonhosted.org/packages/6f/f6/6703fe7cf3d7

In [2]:
import cv2
import numpy as np
import mediapipe as mp

# Configuración de MediaPipe
mp_hands = mp.solutions.hands.Hands(
    static_image_mode=False, 
    max_num_hands=1, 
    min_detection_confidence=0.5, 
    min_tracking_confidence=0.5
)
cap = cv2.VideoCapture(0)  # Iniciar la cámara web

# Almacenamiento de datos y etiquetas
data, labels = [], []

def capture_gesture(label):
    """Captura gestos y guarda los landmarks junto con la etiqueta."""
    print(f"Grabando gesto {label}. Presiona 'q' para detener.")
    
    while True:
        ret, frame = cap.read()
        frame = cv2.flip(frame, 1)  # Refleja la imagen horizontalmente
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = mp_hands.process(frame_rgb)

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                landmarks = np.array([[lm.x, lm.y, lm.z] for lm in hand_landmarks.landmark]).flatten()
                data.append(landmarks)  # Guardar landmarks
                labels.append(label)    # Guardar etiqueta del gesto

                # Dibujar los puntos en la mano detectada
                mp.solutions.drawing_utils.draw_landmarks(
                    frame, hand_landmarks, mp.solutions.hands.HAND_CONNECTIONS
                )

        cv2.imshow('Grabación de Gestos', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

# Captura de gestos con etiquetas 0: Piedra, 1: Papel, 2: Tijeras
for i, gesture in enumerate(["Piedra (0)", "Papel (1)", "Tijeras (2)"]):
    input(f"Presiona Enter para grabar el gesto: {gesture}")
    capture_gesture(i)

# Guardar los datos y etiquetas en archivos .npy
cap.release()
cv2.destroyAllWindows()
np.save('rps_dataset.npy', np.array(data))
np.save('rps_labels.npy', np.array(labels))
print("Dataset guardado con éxito en 'rps_dataset.npy' y 'rps_labels.npy'.")


2024-10-19 17:14:57.176476: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
I0000 00:00:1729368912.739887  546537 gl_context.cc:357] GL version: 2.1 (2.1 ATI-6.1.13), renderer: AMD Radeon Pro 560X OpenGL Engine
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1729368912.764312  547555 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1729368912.782445  547554 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Grabando gesto 0. Presiona 'q' para detener.


2024-10-19 17:15:29.468 Python[9286:546537] +[IMKClient subclass]: chose IMKClient_Legacy
2024-10-19 17:15:29.468 Python[9286:546537] +[IMKInputSession subclass]: chose IMKInputSession_Legacy
W0000 00:00:1729368929.993689  547557 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.
/Users/miri/AAII-2/.venv/lib/python3.10/site-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Grabando gesto 1. Presiona 'q' para detener.
Grabando gesto 2. Presiona 'q' para detener.
Dataset guardado con éxito en 'rps_dataset.npy' y 'rps_labels.npy'.


# train-gesture-classifier

In [3]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split



# Cargar los datos
X = np.load('rps_dataset.npy')
y = np.load('rps_labels.npy')

# División del dataset en entrenamiento y validación
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Definir el modelo con la capa de entrada explícita
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(X.shape[1],)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')  # 3 clases: piedra, papel, tijeras
])

# Compilar el modelo
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

# Entrenar el modelo
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, batch_size=32)

# Guardar el modelo en formato Keras
model.save('rps_model.keras')
print("Modelo entrenado y guardado como 'rps_model.keras'")


Epoch 1/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5600 - loss: 0.9390 - val_accuracy: 0.7924 - val_loss: 0.6150
Epoch 2/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8344 - loss: 0.5271 - val_accuracy: 0.8343 - val_loss: 0.4446
Epoch 3/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8545 - loss: 0.3846 - val_accuracy: 0.8762 - val_loss: 0.3901
Epoch 4/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8721 - loss: 0.3697 - val_accuracy: 0.8902 - val_loss: 0.3394
Epoch 5/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8948 - loss: 0.2972 - val_accuracy: 0.9082 - val_loss: 0.3121
Epoch 6/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9129 - loss: 0.2573 - val_accuracy: 0.9202 - val_loss: 0.2570
Epoch 7/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9356 - loss: 0.2184 - val_accuracy: 0.9242 - val_loss: 0.2523
Epoch 8/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9514 - loss: 0.2003 - val_accuracy: 0.9102 - v

# rock-paper-scissors.

In [4]:
import cv2
import numpy as np
import tensorflow as tf
import mediapipe as mp

# Cargar el modelo entrenado
model = tf.keras.models.load_model('rps_model.keras')

# Inicializar MediaPipe para la detección de manos
mp_hands = mp.solutions.hands.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

# Nombres de las clases
class_names = ['Piedra', 'Papel', 'Tijeras']

# Captura desde la cámara web
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)  # Refleja la imagen para una interacción natural
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = mp_hands.process(frame_rgb)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            landmarks = np.array([[lm.x, lm.y, lm.z] for lm in hand_landmarks.landmark]).flatten()
            
            # Realizar la predicción
            prediction = model.predict(landmarks[np.newaxis, ...])
            class_id = np.argmax(prediction)
            class_name = class_names[class_id]

            # Dibujar los puntos clave y el nombre de la clase
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp.solutions.hands.HAND_CONNECTIONS)
            cv2.putText(frame, class_name, (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Mostrar la imagen con las predicciones
    cv2.imshow('Piedra, Papel o Tijeras', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


I0000 00:00:1729369259.819040  546537 gl_context.cc:357] GL version: 2.1 (2.1 ATI-6.1.13), renderer: AMD Radeon Pro 560X OpenGL Engine
W0000 00:00:1729369259.835357  554535 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1729369259.850571  554535 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step


/Users/miri/AAII-2/.venv/lib/python3.10/site-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━